In [1]:
import pandas as pd

In [68]:
cancer_data = pd.read_csv('/content/drive/MyDrive/Non-Small Cell Lung Cancer_mutation_data.txt', sep = '\t')

In [69]:
print(cancer_data.head())

           SAMPLE_ID PATIENT_ID                 CANCER_TYPE  \
0  P-0000012-T03-IM3  P-0000012  Non-Small Cell Lung Cancer   
1  P-0000012-T03-IM3  P-0000012  Non-Small Cell Lung Cancer   
2  P-0000012-T03-IM3  P-0000012  Non-Small Cell Lung Cancer   
3  P-0000012-T03-IM3  P-0000012  Non-Small Cell Lung Cancer   
4  P-0000012-T03-IM3  P-0000012  Non-Small Cell Lung Cancer   

  CANCER_TYPE_DETAILED Hugo_Symbol              Consequence Reference_Allele  \
0  Lung Adenocarcinoma      PIK3R3         missense_variant                G   
1  Lung Adenocarcinoma         ATR         missense_variant                C   
2  Lung Adenocarcinoma      PDGFRA         missense_variant                T   
3  Lung Adenocarcinoma       FBXW7  splice_acceptor_variant                C   
4  Lung Adenocarcinoma       FBXW7              stop_gained                C   

  Tumor_Seq_Allele1 Tumor_Seq_Allele2 Tumor_Sample_Barcode         HGVSp  
0                 G                 C    P-0000012-T03-IM3   p.Il

In [70]:
most_frequent_symbol = cancer_data['Hugo_Symbol'].value_counts().idxmax()
filtered_cancer_data = cancer_data[(cancer_data['Hugo_Symbol']== most_frequent_symbol)& (cancer_data['Consequence']== 'missense_variant')]


In [71]:
print(cancer_data.shape)
print(filtered_cancer_data.shape)

(66051, 11)
(2760, 11)


In [72]:
filtered_cancer_data.columns

Index(['SAMPLE_ID', 'PATIENT_ID', 'CANCER_TYPE', 'CANCER_TYPE_DETAILED',
       'Hugo_Symbol', 'Consequence', 'Reference_Allele', 'Tumor_Seq_Allele1',
       'Tumor_Seq_Allele2', 'Tumor_Sample_Barcode', 'HGVSp'],
      dtype='object')

In [73]:

cols = [col for col in filtered_cancer_data.columns if col.endswith('_ID') or col in ['Hugo_Symbol','Consequence','HGVSp']]

print(cols)

['SAMPLE_ID', 'PATIENT_ID', 'Hugo_Symbol', 'Consequence', 'HGVSp']


In [74]:
cancer_mut = filtered_cancer_data[cols]


In [75]:
total_mut = len(cancer_mut)
mut_count = cancer_mut['HGVSp'].value_counts()
mut_percentage = ((mut_count/total_mut)*100).round(2)
cancer_mut['mutation_percentage'] = cancer_data['HGVSp'].map(mut_percentage)
print(cancer_mut.head)

<bound method NDFrame.head of                SAMPLE_ID PATIENT_ID Hugo_Symbol       Consequence  \
48     P-0000082-T01-IM3  P-0000082        TP53  missense_variant   
67     P-0000133-T01-IM3  P-0000133        TP53  missense_variant   
71     P-0000165-T01-IM3  P-0000165        TP53  missense_variant   
105    P-0000231-T01-IM3  P-0000231        TP53  missense_variant   
154    P-0000315-T01-IM3  P-0000315        TP53  missense_variant   
...                  ...        ...         ...               ...   
65941  P-0087347-T02-IM7  P-0087347        TP53  missense_variant   
65946  P-0089079-T01-IM7  P-0089079        TP53  missense_variant   
65965  P-0021896-T03-IM7  P-0021896        TP53  missense_variant   
65986  P-0087986-T02-IM7  P-0087986        TP53  missense_variant   
66028  P-0033945-T02-IM7  P-0033945        TP53  missense_variant   

             HGVSp  mutation_percentage  
48     p.His179Arg                 1.27  
67     p.Lys139Asn                 0.14  
71     p.Cys176

<ipython-input-75-c364461ecc0a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cancer_mut['mutation_percentage'] = cancer_data['HGVSp'].map(mut_percentage)


In [76]:
mut_count_table = pd.DataFrame({'mutation_count': mut_count, 'mutation_percentage': mut_percentage})
print(mut_count_table)

             mutation_count  mutation_percentage
HGVSp                                           
p.Val157Phe              87                 3.15
p.Arg158Leu              82                 2.97
p.Arg273Leu              77                 2.79
p.Arg273Cys              56                 2.03
p.Arg175His              56                 2.03
...                     ...                  ...
p.Asn239Lys               1                 0.04
p.Met237Leu               1                 0.04
p.Glu286Gln               1                 0.04
p.Ile255Thr               1                 0.04
p.Val143Gly               1                 0.04

[478 rows x 2 columns]


In [20]:
pip install unipressed

In [27]:
#from unipressed import IdMappingClient

#request = IdMappingClient.submit( source="GeneCards", dest="UniProtKB", ids= {most_frequent_symbol})



In [48]:
ppi_data = pd .read_csv('/content/drive/MyDrive/Dr. V/pioneer_human.txt', sep='\t', header= 0)
print(ppi_data.shape)

(146138, 7)


In [77]:
df_filtered = ppi_data[['uniprot1', 'uniprot2']]

filtered_rows = df_filtered[df_filtered['uniprot1'] == 'Q12888']
print(filtered_rows.shape)

(23, 2)


In [29]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 37.8 MB/s eta 0:00:00


In [78]:
from Bio import ExPASy
from Bio import SwissProt

entry_name = []
gene = []
for num in filtered_rows['uniprot2']:
  record = ExPASy.get_sprot_raw(num)
  record = SwissProt.read(record)
  entry_name.append(record.entry_name)
  gene.append(record.gene_name)
entry_name

['NUD16_HUMAN',
 'TP53B_HUMAN',
 'TIRR_HUMAN',
 'TOPB1_HUMAN',
 'ATM_HUMAN',
 'PLD1_HUMAN',
 'IF16_HUMAN',
 'CJ055_HUMAN',
 'UBAP2_HUMAN',
 'MSL1_HUMAN',
 'GLTL5_HUMAN',
 'SPRE1_HUMAN',
 'OXER1_HUMAN',
 'ANS1A_HUMAN',
 'CSN5_HUMAN',
 'KLD7B_HUMAN',
 'CC120_HUMAN',
 'ARAP1_HUMAN',
 'NEK1_HUMAN',
 'VRK1_HUMAN',
 'RAD18_HUMAN',
 'ASH2L_HUMAN',
 'KMT2B_HUMAN']

In [79]:
filtered_rows['entry_name'] = entry_name
print(filtered_rows)

       uniprot1 uniprot2   entry_name
2900     Q12888   Q96DE0  NUD16_HUMAN
4313     Q12888   Q12888  TP53B_HUMAN
6961     Q12888   Q9BRJ7   TIRR_HUMAN
7333     Q12888   Q92547  TOPB1_HUMAN
104540   Q12888   Q13315    ATM_HUMAN
104541   Q12888   Q13393   PLD1_HUMAN
104542   Q12888   Q16666   IF16_HUMAN
104543   Q12888   Q5SWW7  CJ055_HUMAN
104544   Q12888   Q5T6F2  UBAP2_HUMAN
104545   Q12888   Q68DK7   MSL1_HUMAN
104546   Q12888   Q7Z4T8  GLTL5_HUMAN
104547   Q12888   Q7Z699  SPRE1_HUMAN
104548   Q12888   Q8TDS5  OXER1_HUMAN
104549   Q12888   Q92625  ANS1A_HUMAN
104550   Q12888   Q92905   CSN5_HUMAN
104551   Q12888   Q96G42  KLD7B_HUMAN
104552   Q12888   Q96HB5  CC120_HUMAN
104553   Q12888   Q96P48  ARAP1_HUMAN
104554   Q12888   Q96PY6   NEK1_HUMAN
104555   Q12888   Q99986   VRK1_HUMAN
104556   Q12888   Q9NS91  RAD18_HUMAN
104557   Q12888   Q9UBL3  ASH2L_HUMAN
104558   Q12888   Q9UMN6  KMT2B_HUMAN


<ipython-input-79-2dde520aa0c9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rows['entry_name'] = entry_name


In [16]:
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.1/165.1 kB 5.0 MB/s eta 0:00:00


In [80]:
with pd.ExcelWriter('/content/drive/MyDrive/Dr. V/Non-Small Cell Lung_cancer_mut.xlsx', engine='xlsxwriter') as writer:
    cancer_mut.to_excel(writer, sheet_name='mutations', index=False)
    mut_count_table.to_excel(writer, sheet_name='AA_change_freq%', index=True)
    filtered_rows.to_excel(writer, sheet_name="Pioneer_PPI's", index=False)